In [1]:
import sqlite3
import numpy as np
import pandas as pd
import time
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)

db = sqlite3.connect('L21DB.sqlite')
cursor = db.cursor()
cursor.execute("DROP TABLE IF EXISTS candidates")
cursor.execute("DROP TABLE IF EXISTS contributors")
cursor.execute("PRAGMA foreign_keys=1")

cursor.execute('''CREATE TABLE candidates (
               id INTEGER PRIMARY KEY NOT NULL, 
               first_name TEXT, 
               last_name TEXT, 
               middle_init TEXT, 
               party TEXT NOT NULL)''')

db.commit() # Commit changes to the database

cursor.execute('''CREATE TABLE contributors (
          id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, 
          last_name TEXT, 
          first_name TEXT, 
          middle_name TEXT, 
          street_1 TEXT, 
          street_2 TEXT, 
          city TEXT, 
          state TEXT, 
          zip TEXT, 
          amount REAL, 
          date DATETIME, 
          candidate_id INTEGER NOT NULL, 
          FOREIGN KEY(candidate_id) REFERENCES candidates(id))''')

db.commit()

with open ("candidates.txt") as candidates:
    next(candidates) # jump over the header
    for line in candidates.readlines():
        cid, first_name, last_name, middle_name, party = line.strip().split('|')
        vals_to_insert = (int(cid), first_name, last_name, middle_name, party)
        cursor.execute('''INSERT INTO candidates 
                  (id, first_name, last_name, middle_init, party)
                  VALUES (?, ?, ?, ?, ?)''', vals_to_insert)

with open ("contributors.txt") as contributors:
    next(contributors)
    for line in contributors.readlines():
        cid, last_name, first_name, middle_name, street_1, street_2, \
            city, state, zip_code, amount, date, candidate_id = line.strip().split('|')
        vals_to_insert = (last_name, first_name, middle_name, street_1, street_2, 
                          city, state, int(zip_code), amount, date, candidate_id)
        cursor.execute('''INSERT INTO contributors (last_name, first_name, middle_name, 
                           street_1, street_2, city, state, zip, amount, date, candidate_id) 
                           VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''', vals_to_insert)

candidate_cols = [col[1] for col in cursor.execute("PRAGMA table_info(candidates)")]
contributor_cols = [col[1] for col in cursor.execute("PRAGMA table_info(contributors)")]

def viz_tables(cols, query):
    q = cursor.execute(query).fetchall()
    framelist = dict()
    for i, col_name in enumerate(cols):
        framelist[col_name] = [col[i] for col in q]
    return pd.DataFrame.from_dict(framelist)

### =====================================================================================================
### SQL EXERCISES SQL EXERCISES SQL EXERCISES


### INNER JOIN

# 1. inner join the candidates and contributors table to show which contributors support which candidates
inner_join = '''SELECT contributors.last_name, contributors.first_name, candidates.last_name
                FROM contributors INNER JOIN candidates ON contributors.candidate_id = candidates.id'''
display(viz_tables(['last_name', 'first_name', 'candidate_last_name'], inner_join))

# 2. inner join the candidates and contributors table to show which contributors supported Obama
inner_join_obama = '''SELECT contributors.last_name, contributors.first_name, candidates.last_name
                      FROM contributors INNER JOIN candidates ON contributors.candidate_id = candidates.id
                      WHERE candidates.last_name = "Obama"'''
display(viz_tables(['last_name', 'first_name', 'candidate_last_name'], inner_join_obama))



### LEFT JOIN / LEFT OUTER JOIN

# 1./2. see tables in second Markdown cell

# 3. left join to see the average contribution towards a candidate, and the number of contributors for 
# that candidate
left_join = '''SELECT AVG(contributors.amount), COUNT(contributors.last_name), candidates.last_name 
               FROM contributors LEFT JOIN candidates ON contributors.candidate_id = candidates.id 
               GROUP BY candidates.last_name'''
display(viz_tables(['average contribution', 'number of contributors', 'candidate last name'], left_join))

### =====================================================================================================

### PANDAS PANDAS PANDAS PANDAS PANDAS PANDAS PANDAS PANDAS PANDAS PANDAS PANDAS 

# creating a new database
dbp = sqlite3.connect('L21_pandas_DB.sqlite')
csr = dbp.cursor()
csr.execute("DROP TABLE IF EXISTS candidates")
csr.execute("DROP TABLE IF EXISTS contributors")
csr.execute("PRAGMA foreign_keys=1")

csr.execute('''CREATE TABLE candidates (
               id INTEGER PRIMARY KEY NOT NULL, 
               first_name TEXT, 
               last_name TEXT, 
               middle_name TEXT, 
               party TEXT NOT NULL)''')

dbp.commit() # Commit changes to the database

csr.execute('''CREATE TABLE contributors (
          id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, 
          last_name TEXT, 
          first_name TEXT, 
          middle_name TEXT, 
          street_1 TEXT, 
          street_2 TEXT, 
          city TEXT, 
          state TEXT, 
          zip TEXT, 
          amount REAL, 
          date DATETIME, 
          candidate_id INTEGER NOT NULL, 
          FOREIGN KEY(candidate_id) REFERENCES candidates(id))''')

dbp.commit()

# new viz tables function for our new database
def viz_tables_csr(cols, query):
    q = csr.execute(query).fetchall()
    framelist = dict()
    for i, col_name in enumerate(cols):
        framelist[col_name] = [col[i] for col in q]
    return pd.DataFrame.from_dict(framelist)

# reading in contributors csv into pandas dataframe (done previously)
dfcontr = pd.read_csv("contributors.txt", sep="|")
dfcand = pd.read_csv("candidates.txt", sep="|")
dfcand.to_sql("candidates", dbp, if_exists="append", index=False)

### =====================================================================================================
### PANDAS EXERCISES PANDAS EXERCISES PANDAS EXERCISES


## POPULATING TABLE / QUERYING IN PANDAS

# 1. Populating the contributors table in the dbp database
dfcontr.to_sql("contributors", dbp, if_exists="append", index=False)
query = '''SELECT * FROM contributors'''
display(viz_tables_csr(contributor_cols, query))

# 2A. Query for state == "VA" and amount is less than 400 from contributors
display(dfcontr[(dfcontr.state == "VA") & (dfcontr.amount < 400)])

# 2B. Query for state is "NULL"
display(dfcontr[dfcontr.state.isnull()])

# 2C. Query for states of TX and PA
display(dfcontr.query("state == 'TX' | state == 'PA'"))

# 2D. Query for amount contributed between 10 and 50
display(dfcontr.query("amount >= 10 & amount <= 50"))



## SORTING / SELECTING IN PANDAS

# 1. Sorting contributors table by amount in descending order
display(dfcontr.sort_values(by='amount', ascending=False))

# 2. Selecting first_name and amount columns
display(dfcontr[['first_name', 'amount']])

# 3. Selecting last_name and first_name columns and drop duplicates
last_first_drop_dup = dfcontr[['first_name', 'last_name']].drop_duplicates()
display(last_first_drop_dup)

# 4. Count how many there are after duplicates have been dropped
display(last_first_drop_dup.count())



## ALTERING IN PANDAS

# 1. new "last name, first name" column
dfcontr['name'] = dfcontr['last_name'] + ", " + dfcontr['first_name']
display(dfcontr)

# 2. PA contributors name change to "X"
dfcontr.loc[dfcontr.state == "PA", "name"] = "X"
display(dfcontr.query("state == 'PA'"))

# 3. Deleting new name column
del dfcontr['name']
display(dfcontr)



## AGGREGATION IN PANDAS

# Describing the contributors table
display(dfcontr.describe())

### END EXERCISES

db.commit()
db.close()
dbp.commit()
dbp.close()

,last_name,first_name,candidate_last_name
0,Agee,Steven,Huckabee
1,Ahrens,Don,Huckabee
2,Ahrens,Don,Huckabee
3,Ahrens,Don,Huckabee
4,Akin,Charles,Huckabee
5,Akin,Mike,Huckabee
6,Akin,Rebecca,Huckabee
7,Aldridge,Brittni,Huckabee
8,Allen,John D.,Huckabee
9,Allen,John D.,Huckabee


,last_name,first_name,candidate_last_name
0,Buckler,Steve,Obama
1,Buckler,Steve,Obama
2,Buckheit,Bruce,Obama
3,Buckel,Linda,Obama
4,Buckel,Linda,Obama
5,Buckel,Linda,Obama
6,Buck,Thomas,Obama
7,Buck,Jay,Obama
8,Buck,Blaine,Obama
9,Buck,Barbara,Obama


,average contribution,number of contributors,candidate last name
0,50.6000,25,Clinton
1,-1360.0000,25,Giuliani
2,772.0000,25,Huckabee
3,452.6000,25,McCain
4,472.4356,25,Obama
5,-519.7088,25,Paul
6,156.0000,25,Romney


,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,1,Agee,Steven,None,549 Laurel Branch Road,None,Floyd,VA,24091,500.0,2007-06-30,16
1,2,Ahrens,Don,None,4034 Rennellwood Way,None,Pleasanton,CA,94566,250.0,2007-05-16,16
2,3,Ahrens,Don,None,4034 Rennellwood Way,None,Pleasanton,CA,94566,50.0,2007-06-18,16
3,4,Ahrens,Don,None,4034 Rennellwood Way,None,Pleasanton,CA,94566,100.0,2007-06-21,16
4,5,Akin,Charles,None,10187 Sugar Creek Road,None,Bentonville,AR,72712,100.0,2007-06-16,16
5,6,Akin,Mike,None,181 Baywood Lane,None,Monticello,AR,71655,1500.0,2007-05-18,16
6,7,Akin,Rebecca,None,181 Baywood Lane,None,Monticello,AR,71655,500.0,2007-05-18,16
7,8,Aldridge,Brittni,None,"808 Capitol Square Place, SW",None,Washington,DC,20024,250.0,2007-06-06,16
8,9,Allen,John D.,None,1052 Cannon Mill Drive,None,North Augusta,SC,29860,1000.0,2007-06-11,16
9,10,Allen,John D.,None,1052 Cannon Mill Drive,None,North Augusta,SC,29860,1300.0,2007-06-29,16


,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
27,NaN,Buckheit,Bruce,NaN,8904 KAREN DR,NaN,FAIRFAX,VA,220312731,100.00,2007-09-19,20
77,NaN,Ranganath,Anoop,NaN,2507 Willard Drive,NaN,Charlottesville,VA,22903,-100.00,2008-04-21,32
88,NaN,Perreault,Louise,NaN,503 Brockridge Hunt Drive,NaN,Hampton,VA,23666,-34.08,2008-04-21,32
145,NaN,ABDELLA,THOMAS,M.,4231 MONUMENT WALL WAY #340,NaN,FAIRFAX,VA,220308440,50.00,2007-09-30,35


...

first_name    126
last_name     126
dtype: int64

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id,name
0,NaN,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500.0,2007-06-30,16,"Agee, Steven"
1,NaN,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,250.0,2007-05-16,16,"Ahrens, Don"
2,NaN,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,50.0,2007-06-18,16,"Ahrens, Don"
3,NaN,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,100.0,2007-06-21,16,"Ahrens, Don"
4,NaN,Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100.0,2007-06-16,16,"Akin, Charles"
5,NaN,Akin,Mike,NaN,181 Baywood Lane,NaN,Monticello,AR,71655,1500.0,2007-05-18,16,"Akin, Mike"
6,NaN,Akin,Rebecca,NaN,181 Baywood Lane,NaN,Monticello,AR,71655,500.0,2007-05-18,16,"Akin, Rebecca"
7,NaN,Aldridge,Brittni,NaN,"808 Capitol Square Place, SW",NaN,Washington,DC,20024,250.0,2007-06-06,16,"Aldridge, Brittni"
8,NaN,Allen,John D.,NaN,1052 Cannon Mill Drive,NaN,North Augusta,SC,29860,1000.0,2007-06-11,16,"Allen, John D."
9,NaN,Allen,John D.,NaN,1052 Cannon Mill Drive,NaN,North Augusta,SC,29860,1300.0,2007-06-29,16,"Allen, John D."


,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id,name
70,NaN,BUCKLEY,WALTER,W.,1635 COUNTRY ROAD,NaN,BETHLEHEM,PA,180155718,-100.0,2008-03-05,22,X
71,NaN,BUCKLEY,MARJORIE,B.,1635 COUNTRY ROAD,NaN,BETHLEHEM,PA,180155718,-100.0,2008-03-05,22,X
93,NaN,Raught,Philip,M,4714 Plum Way,NaN,Pittsburgh,PA,15201,-1046.0,2008-04-21,32,X
94,NaN,Ferrara,Judith,D,1508 Waterford Road,NaN,Yardley,PA,19067,-1100.0,2008-04-21,32,X
165,NaN,ABEL,JOHN,H.,422 THOMAS STREET,NaN,BETHLEHEM,PA,180153316,200.0,2008-01-22,37,X


,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,NaN,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500.0,2007-06-30,16
1,NaN,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,250.0,2007-05-16,16
2,NaN,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,50.0,2007-06-18,16
3,NaN,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,100.0,2007-06-21,16
4,NaN,Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100.0,2007-06-16,16
5,NaN,Akin,Mike,NaN,181 Baywood Lane,NaN,Monticello,AR,71655,1500.0,2007-05-18,16
6,NaN,Akin,Rebecca,NaN,181 Baywood Lane,NaN,Monticello,AR,71655,500.0,2007-05-18,16
7,NaN,Aldridge,Brittni,NaN,"808 Capitol Square Place, SW",NaN,Washington,DC,20024,250.0,2007-06-06,16
8,NaN,Allen,John D.,NaN,1052 Cannon Mill Drive,NaN,North Augusta,SC,29860,1000.0,2007-06-11,16
9,NaN,Allen,John D.,NaN,1052 Cannon Mill Drive,NaN,North Augusta,SC,29860,1300.0,2007-06-29,16


,id,zip,amount,candidate_id
count,0.0,1.750000e+02,175.000000,175.000000
mean,NaN,3.780014e+08,3.418114,28.000000
std,NaN,3.628278e+08,1028.418999,7.823484
min,NaN,2.474000e+03,-2592.000000,16.000000
25%,NaN,9.336700e+04,-175.000000,20.000000
50%,NaN,3.233313e+08,100.000000,32.000000
75%,NaN,7.816946e+08,300.000000,35.000000
max,NaN,9.951532e+08,4600.000000,37.000000


### DRAW TABLE EXERCISES FOR LEFT JOIN / LEFT OUTER JOIN

**1. LEFT JOIN using Table A as reference and id columns for comparison**

| A.nA   | A.attr  | B.nB |  B.attr
| :::: | ::::: | ::: | :::
| s1   | 23    | t1   | 60
| s2   | 7     | t3   | 22
| s3   | 15    | t3   | 22
| s4   | 31    | t1   | 60

**2. LEFT JOIN using Table B as reference and id columns for comparison**

| B.nB |  B.attr| A.nA   | A.attr  
| :::: | ::::: | ::: | :::
| t1   | 60| s1   | 23    
| t1   | 60| s4   | 31    
| t2   | 14| None | NaN   
| t3   | 22| s2   | 7     
| t3   | 22| s3   | 15    